In [1]:
# Setup
%pip install timm==0.6.7
# !git clone https://github.com/isl-org/ZoeDepth.git
# !cd ZoeDepth && python sanity.py

# Append ZoeDepth to path
import sys
sys.path.append('ZoeDepth')

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports
from zoedepth.utils.misc import colorize
import matplotlib.pyplot as plt
import cv2
import torch
import pathlib
import os
from tqdm import tqdm
import numpy as np

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent.parent
DATA_DIR = GIT_ROOT / "data" / 'ICMI2024'

In [3]:
# References: 
# https://huggingface.co/spaces/shariqfarooq/ZoeDepth/blob/962e87ab4eb0e419610ddf512841dec2558dcf7c/geometry.py#L3
# Load the models
torch.hub.help("intel-isl/MiDaS", "DPT_BEiT_L_384", force_reload=True)  # Triggers fresh download of MiDaS repo
repo = "isl-org/ZoeDepth"

##### Select model
# Zoe_N
# model_zoe_n = torch.hub.load(repo, "ZoeD_N", pretrained=True)

# # Zoe_K
# model_zoe_k = torch.hub.load(repo, "ZoeD_K", pretrained=True)

# Zoe_NK
model_zoe_nk = torch.hub.load(repo, "ZoeD_NK", pretrained=True)

##### sample prediction
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
zoe = model_zoe_nk.to(DEVICE)


Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /home/nicole/.cache/torch/hub/master.zip
Using cache found in /home/nicole/.cache/torch/hub/isl-org_ZoeDepth_main


img_size [384, 512]


Using cache found in /home/nicole/.cache/torch/hub/intel-isl_MiDaS_master
/home/nicole/anaconda3/envs/ettk_dev/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Params passed to Resize transform:
	width:  512
	height:  384
	resize_target:  True
	keep_aspect_ratio:  True
	ensure_multiple_of:  32
	resize_method:  minimal
Using pretrained resource url::https://github.com/isl-org/ZoeDepth/releases/download/v1.0/ZoeD_M12_NK.pt
Loaded successfully


In [7]:
RECORD = True

def process(vid_file, depth_output_file):

    # Skip if depth file exists
    if depth_output_file.exists():
        return

    # Load video
    assert vid_file.exists()
    cap = cv2.VideoCapture(str(vid_file))
    fps = cap.get(cv2.CAP_PROP_FPS)
    LENGTH = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # LENGTH = 10 

    # Reset
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    if RECORD:
        # Define the codec and create VideoWriter object
        output_file = DATA_DIR / 'rgbd.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        # out = cv2.VideoWriter(str(output_file), fourcc, fps, (1920*2, 1080))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out_depth = cv2.VideoWriter(str(depth_output_file), fourcc, fps, (1920, 1080), isColor=False)

    for i in tqdm(range(LENGTH), total=LENGTH):

        # Load frame
        ret, frame = cap.read()
        if not ret:
            break
        
        # Generate depth
        depth = zoe.infer_pil(frame) # (1080,1920)
        colored_depth = colorize(depth) # [h,w,4]

        # Stich together
        img = cv2.hconcat([frame, colored_depth[:,:,:3]])

        cv2.imshow('frame', img)
        # cv2.imshow('frame', frame)
        # cv2.imshow('depth', colored_depth)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if RECORD:
            # out.write(img)
            depth_norm = cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
            out_depth.write(depth_norm)

    cv2.destroyAllWindows()
    if RECORD:
        # out.release()
        out_depth.release()

files = [
    # Camera 1
    # [DATA_DIR / 'videos' / 'day1' / "block-a-blue-day1-first-group-cam1.mp4", DATA_DIR / 'depths' / 'day1' / 'group1-depth-cam1.mp4'],
    # [DATA_DIR / 'videos' / 'day1' / "block-a-blue-day1-second-group-cam1.mp4", DATA_DIR / 'depths' / 'day1' / 'group2-depth-cam1.mp4'],
    # [DATA_DIR / 'videos' / 'day2' / "block-a-blue-day2-first-group-cam1.mp4", DATA_DIR / 'depths' / 'day2' / 'group1-depth-cam1.mp4'],
    # [DATA_DIR / 'videos' / 'day2' / "block-a-blue-day2-second-group-cam1.mp4", DATA_DIR / 'depths' / 'day2' / 'group2-depth-cam1.mp4']

    # Camera 2
    # [DATA_DIR / 'videos' / 'day1' / "block-a-blue-day1-first-group-cam2.mp4", DATA_DIR / 'depths' / 'day1' / 'group1-depth-cam2.mp4'],
    # [DATA_DIR / 'videos' / 'day1' / "block-a-blue-day1-second-group-cam2.mp4", DATA_DIR / 'depths' / 'day1' / 'group2-depth-cam2.mp4'],
    [DATA_DIR / 'videos' / 'day2' / "block-a-blue-day2-first-group-cam2.mp4", DATA_DIR / 'depths' / 'day2' / 'group1-depth-cam2.mp4'],
    [DATA_DIR / 'videos' / 'day2' / "block-a-blue-day2-second-group-cam2.mp4", DATA_DIR / 'depths' / 'day2' / 'group2-depth-cam2.mp4']
]

for (vid, depth) in files:
    process(vid, depth)

100%|██████████| 18834/18834 [1:48:41<00:00,  2.89it/s]
